In [1]:
import numpy as np
import pandas as pd
import math
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [2]:
mod = SourceModule("""
    #define N 1000
    
    
  
    
    
    __global__ void doIndexy(float *Z, int *indexx, int *indexy, float *pin, float *PYX,int total )
    { 
       
        int idx = blockIdx.x * blockDim.x + threadIdx.x; //this is actually j in indexy
        int t = 0; 
        if(idx >= total){
            return;
        }
        //printf("%d/n", total);
        float sum = 0;
        int j = indexy[idx]; 
        for (int i = 0; i < total ;i++){
           //if(Z[indexx[i]+j]!=0.f){
               sum += Z[indexx[i]+j]*log2(Z[indexx[i]+j]);
           //}
           
           printf("%d\\n", Z[0]);
        }; 
        //printf("%f/n", sum);
        PYX[idx] = sum; 
        
        //printf ("%d:::::::\\t", idx);
       
       // in[idx*(d+1)+5]=integral(in[idx*(d+1)+0],in[idx*(d+1)+1], in[idx*(d+1)+2], in[idx*(d+1)+3], in[idx*(d+1)+4]);
    }""")


/home/shahrezaeima/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(13): warning: variable "t" was declared but never referenced




In [3]:
func = mod.get_function("doIndexy")


In [4]:
Z = np.random.randn(25,3)
Z = np.abs(Z)

In [5]:
Z

array([[ 0.99813463,  0.86093625,  1.9598039 ],
       [ 1.18013923,  0.25324364,  0.87328898],
       [ 0.08020349,  1.51911916,  1.38996076],
       [ 0.85149162,  1.62186049,  0.59641299],
       [ 1.08906999,  1.29938762,  0.22550147],
       [ 1.42626013,  0.64179578,  0.01514069],
       [ 1.26575828,  0.55933667,  1.92368534],
       [ 0.05316282,  2.09836495,  0.64975999],
       [ 0.55214801,  1.18547271,  0.16522891],
       [ 0.27231505,  1.46899591,  1.26272702],
       [ 0.2799207 ,  1.41480321,  1.04845131],
       [ 1.14918285,  0.12141603,  1.65641143],
       [ 0.3961384 ,  0.47977233,  0.67813919],
       [ 0.86923015,  1.68140873,  2.11712326],
       [ 0.31432721,  0.4943301 ,  0.33686418],
       [ 1.55365251,  0.74529244,  0.42242865],
       [ 0.98248439,  0.16859014,  0.75575313],
       [ 0.04003684,  0.17619586,  1.39815884],
       [ 0.57649919,  0.41530866,  1.74095005],
       [ 1.43885911,  0.71787172,  3.73404045],
       [ 1.27636285,  0.30848062,  0.807

In [6]:
def in_dist(a,b,tau=1):
    ps = np.empty(len(a))
    for i in range(len(a)):
        if(b[i] >= a[i]):
            ps[i] = (1/tau**2)*(math.exp(-1*b[i]/tau))
        else: 
            ps[i] = 0
    return ps
            

In [7]:
# x, y = np.meshgrid([x for x in np.arange(0,2.05,0.05)],[x for x in np.arange(0,2.05,0.05)])
x, y = np.meshgrid([x for x in np.arange(0,2,1)],[x for x in np.arange(0,2,1)])

In [8]:
t = y 
y = np.reshape(x,-1)
x =np.reshape(t,-1)

In [9]:
pin = in_dist(x,y, 0.5)
pin = pin.T

In [10]:
pin

array([ 4.        ,  0.54134113,  0.        ,  0.54134113])

In [11]:
indexy = np.array([],dtype=int)
for k in range (1,3):
    
    a = [i for i in range(1+5*(k-1),3+5*(k-1))] ### possible index error
    
    indexy =np.append(indexy,a)


In [12]:
indexx = np.array([],dtype=int)
for k in range (0,4):
    
    a = [i for i in range(5*(k),5*(k)+4)] ### possible index error
    print (a)
    indexx =np.append(indexx,a)

[0, 1, 2, 3]
[5, 6, 7, 8]
[10, 11, 12, 13]
[15, 16, 17, 18]


In [13]:
indexx = indexx-1
indexx+indexy[-1]

array([ 6,  7,  8,  9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24])

In [14]:
indexy

array([1, 2, 6, 7])

In [15]:
indexx

array([-1,  0,  1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 14, 15, 16, 17])

In [16]:
PYX = np.empty(len(indexy),np.float32)
PY = np.empty(len(indexx),np.float32)

In [17]:
count = 0
for j in indexy:
    A = Z[indexx+j,2]
    PYX [count] = 0.05*0.05*np.sum(A*np.log(A))
    count = count + 1

In [18]:
count = 0
for j in indexx:
    B = Z[indexy+j,2]
    PY [count] = 0.05*0.05*np.sum(B*pin)
    count = count + 1

In [19]:
PYX

array([ 0.0125888 ,  0.02151615,  0.00901642,  0.02435944], dtype=float32)

In [20]:
PY

array([ 0.02338333,  0.01149335,  0.01493038,  0.00797823,  0.00499654,
        0.02103397,  0.00958643,  0.0038171 ,  0.01374902,  0.01937408,
        0.01200273,  0.0266806 ,  0.00737215,  0.00976739,  0.01652697,
        0.02461668], dtype=float32)

In [21]:
Z_2 = Z[:,2]

array([ 1.95980394,  0.87328899,  1.38996077,  0.59641302,  0.22550146,
        0.01514069,  1.92368531,  0.64976001,  0.16522892,  1.26272702,
        1.0484513 ,  1.65641141,  0.67813921,  2.11712337,  0.33686417,
        0.42242864,  0.75575316,  1.39815879,  1.74095011,  3.7340405 ,
        0.80783093,  1.57021999,  0.23472033,  0.13984677,  1.59138346], dtype=float32)

In [22]:
d_Z_2 = cuda.mem_alloc(np.float32(Z_2).nbytes)
cuda.memcpy_htod(d_Z_2, np.float32(Z_2))

d_indexx = cuda.mem_alloc(np.int32(indexx).nbytes)
cuda.memcpy_htod(d_indexx, np.int32(indexx))

d_indexy = cuda.mem_alloc(np.int32(indexy).nbytes)
cuda.memcpy_htod(d_indexy, np.int32(indexy))

d_pin = cuda.mem_alloc(np.float32(pin).nbytes) 
cuda.memcpy_htod(d_pin, np.float32(pin))

d_PYX = cuda.mem_alloc(np.float32(PYX).nbytes) 
cuda.memcpy_htod(d_PYX, np.float32(PYX))




In [23]:
blocksize = 4
gridsize = math.floor(len(indexy)/blocksize)+1
func(d_Z_2, d_indexx, d_indexy, d_pin, d_PYX, np.int32(len(indexx)),  block=(blocksize,1,1), grid =(gridsize,1,1))

In [24]:
len(indexx)

16

In [25]:
h_test_out = np.zeros(len(indexy), np.float32)




In [26]:
cuda.memcpy_dtoh(h_test_out, d_PYX)

In [27]:
h_test_out


array([  7.26471806,  12.41649532,   5.20317841,  14.05729675], dtype=float32)

In [28]:
pd.DataFrame(h_test_out)

0
0   7.264718
1  12.416495
2   5.203178
3  14.057297